<a href="https://colab.research.google.com/github/namanphy/END2/blob/main/S4/Basic_LSTM_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 197k/84.1M [00:00<00:52, 1.58MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 75.6MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['THE', 'MATADOR', 'is', 'hit', '-', 'man', 'movie', 'lite', '....', 'if', 'you', 'can', 'say', 'that', 'about', 'a', 'hit', '-', 'man', 'movie', '.', 'The', 'violence', 'is', 'never', 'really', 'shown', 'but', 'often', 'introduced', '.', 'At', 'first', 'I', 'was', 'scared', 'I', 'was', 'in', 'for', 'another', 'retread', 'of', 'mid-90s', 'gangster', '-', 'hit', '-', 'man', '-', 'hipster', '-', 'dark', 'comedy', 'BUT', 'was', 'happily', 'surprised', 'when', 'I', 'realized', 'this', 'is', 'just', 'a', 'sweet', 'and', 'humorous', 'story', 'about', 'friendship', '.', 'Nothing', 'terribly', 'exciting', 'happens', 'in', 'this', 'film', 'but', 'every', 'bit', 'of', 'it', 'is', 'kept', 'me', 'grinning', '.', 'The', 'three', 'leads', 'have', 'the', 'best', 'chemistry', 'the', 'big', 'screen', 'has', 'offered', 'in', 'recent', 'years', 'and', 'it', 'looks', 'like', 'they', 'had', 'a', 'great', 'time', 'making', 'this', 'film', 'together', '.', 'The', 'writing', 'is', 'sharp', 'though', 

In [5]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [13]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [14]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [15]:
# Most frequent words
print(TEXT.vocab.freqs.most_common(20))

[('the', 202276), (',', 192266), ('.', 165263), ('and', 109304), ('a', 109236), ('of', 100449), ('to', 93376), ('is', 76310), ('in', 61193), ('I', 53947), ('it', 53513), ('that', 49067), ('"', 44784), ("'s", 43183), ('this', 42108), ('-', 37196), ('/><br', 35665), ('was', 34713), ('as', 30320), ('with', 29881)]


In [16]:
# Interger to strings

print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [21]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


## Creating Iterators

In [22]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

## Building Model


### Using LSTM's hidden state to pass through FC for final output.

In [76]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        # self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc1 = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.ReLU()
        )
        self.fc2 = nn.Linear(128, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        # output, hidden = self.rnn(embedded)
        output, (hidden, cell_state) = self.lstm(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]

        # print('ALRIGHT TILL HERE')
        # print(hidden)

        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc2(self.fc1(hidden.squeeze(0)))

In [77]:
# Specifying Dimensions
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

# Model Instantiation
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [78]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,899,817 trainable parameters


In [79]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [80]:
criterion = nn.BCEWithLogitsLoss()

In [81]:
model = model.to(device)
criterion = criterion.to(device)

In [41]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

## Train / Test loop

In [42]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [44]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [82]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 25s
	Train Loss: 0.693 | Train Acc: 50.36%
	 Val. Loss: 0.693 |  Val. Acc: 49.63%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.38%
	 Val. Loss: 0.693 |  Val. Acc: 49.74%
Epoch: 03 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.34%
	 Val. Loss: 0.693 |  Val. Acc: 49.92%
Epoch: 04 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.34%
	 Val. Loss: 0.693 |  Val. Acc: 50.07%
Epoch: 05 | Epoch Time: 0m 26s
	Train Loss: 0.693 | Train Acc: 50.34%
	 Val. Loss: 0.693 |  Val. Acc: 50.08%


In [70]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.698 | Test Acc: 50.29%


### Better accuracy over RNN - if passing the final hidden state with two fully connected layers.

In [83]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.693 | Test Acc: 52.39%


### **A little loss over RNN.**